# Caso generico

In [1]:
import numpy as np

# Estados: [frecuencia cardiaca, glucosa]
# Acciones: 0 = no hacer nada, 1 = dar medicación, 2 = recomendar ejercicio
# Recompensa: mejora en las métricas

n_states = 100  # discretización de posibles estados
n_actions = 3
Q = np.zeros((n_states, n_actions))

# Simulación
for episode in range(500):
    state = np.random.randint(0, n_states)
    done = False

    while not done:
        action = np.argmax(Q[state] + np.random.randn(1, n_actions) * 0.1)

        # Simulamos transición y recompensa
        if action == 1:  # medicación
            reward = 2
        elif action == 2:  # ejercicio
            reward = 1
        else:
            reward = -1

        next_state = (state + np.random.randint(-3, 3)) % n_states

        # Actualización Q-learning
        Q[state, action] += 0.1 * (reward + 0.9 * np.max(Q[next_state]) - Q[state, action])

        state = next_state
        done = np.random.rand() < 0.1  # 10% de probabilidad de terminar

print("Tabla Q final:")
print(Q)


Tabla Q final:
[[ 0.          7.33901672  0.        ]
 [ 0.          0.          5.70104457]
 [ 0.          6.42663646  0.        ]
 [-0.1         7.0239987   0.        ]
 [ 0.          5.98365536  0.118     ]
 [ 0.          6.76282459  0.        ]
 [-0.1         0.          5.93697511]
 [-0.064342    7.01500636  0.        ]
 [ 0.          6.51251366  0.12173952]
 [ 0.          6.4429814   0.        ]
 [ 0.          0.          5.36035246]
 [ 0.          0.          4.89208927]
 [-0.091       0.          4.52897726]
 [ 0.          5.06721422  0.1       ]
 [-0.1         0.          4.26665869]
 [-0.1         4.8399159   0.        ]
 [-0.091       0.          4.50925444]
 [-0.082       5.73506884  0.        ]
 [-0.1         5.76632026  0.        ]
 [-0.04505463  0.          5.23498066]
 [ 0.          5.98488272  0.        ]
 [ 0.          6.16627896  0.        ]
 [ 0.          5.4743797   0.1       ]
 [ 0.          0.          4.10606027]
 [ 0.          0.26622632  4.299631  ]
 [-0.05122

# Caso diabetes

In [2]:
import numpy as np

# Parámetros del entorno
niveles_glucosa = np.linspace(50, 300, 26)  # 26 estados (mg/dL)
acciones = [-2, -1, 0, 1, 2]  # Cambio en dosis (-2 = mucho menos, 2 = mucho más)
n_states = len(niveles_glucosa)
n_actions = len(acciones)

# Tabla Q
Q = np.zeros((n_states, n_actions))

# Función de recompensa
def recompensa(glucosa):
    if 80 <= glucosa <= 120:  # rango óptimo
        return 10
    elif 60 <= glucosa <= 140:
        return 5
    else:
        return -10

# Simulación de entrenamiento
for episodio in range(200):
    estado = np.random.randint(0, n_states)  # glucosa inicial
    glucosa_actual = niveles_glucosa[estado]
    terminado = False

    while not terminado:
        # Política ε-greedy
        if np.random.rand() < 0.1:
            accion_idx = np.random.randint(0, n_actions)
        else:
            accion_idx = np.argmax(Q[estado])

        cambio_dosis = acciones[accion_idx]

        # Simulación de efecto en glucosa
        glucosa_siguiente = glucosa_actual - cambio_dosis * np.random.randint(5, 15)
        glucosa_siguiente = np.clip(glucosa_siguiente, 50, 300)

        # Obtener nuevo estado y recompensa
        nuevo_estado = np.argmin(abs(niveles_glucosa - glucosa_siguiente))
        r = recompensa(glucosa_siguiente)

        # Actualización Q-learning
        Q[estado, accion_idx] += 0.1 * (r + 0.9 * np.max(Q[nuevo_estado]) - Q[estado, accion_idx])

        estado = nuevo_estado
        glucosa_actual = glucosa_siguiente

        # Condición de finalización
        terminado = np.random.rand() < 0.05  # fin de episodio aleatorio

# Política aprendida
politica = [acciones[np.argmax(Q[s])] for s in range(n_states)]

print("Política aprendida (cambio de dosis por nivel de glucosa):")
for g, a in zip(niveles_glucosa, politica):
    print(f"Glucosa {g:.0f} mg/dL -> cambio de dosis {a}")


Política aprendida (cambio de dosis por nivel de glucosa):
Glucosa 50 mg/dL -> cambio de dosis -2
Glucosa 60 mg/dL -> cambio de dosis -2
Glucosa 70 mg/dL -> cambio de dosis -2
Glucosa 80 mg/dL -> cambio de dosis -2
Glucosa 90 mg/dL -> cambio de dosis -2
Glucosa 100 mg/dL -> cambio de dosis -2
Glucosa 110 mg/dL -> cambio de dosis 0
Glucosa 120 mg/dL -> cambio de dosis -1
Glucosa 130 mg/dL -> cambio de dosis 1
Glucosa 140 mg/dL -> cambio de dosis 1
Glucosa 150 mg/dL -> cambio de dosis 2
Glucosa 160 mg/dL -> cambio de dosis 2
Glucosa 170 mg/dL -> cambio de dosis 2
Glucosa 180 mg/dL -> cambio de dosis 2
Glucosa 190 mg/dL -> cambio de dosis 2
Glucosa 200 mg/dL -> cambio de dosis 2
Glucosa 210 mg/dL -> cambio de dosis -1
Glucosa 220 mg/dL -> cambio de dosis 2
Glucosa 230 mg/dL -> cambio de dosis -1
Glucosa 240 mg/dL -> cambio de dosis 1
Glucosa 250 mg/dL -> cambio de dosis -1
Glucosa 260 mg/dL -> cambio de dosis 1
Glucosa 270 mg/dL -> cambio de dosis -1
Glucosa 280 mg/dL -> cambio de dosis -

#  Control de ventilador mecánico con Q-Learning

In [4]:
import numpy as np

# Parámetros
n_states = 21  # Saturación de oxígeno: 80% a 100% (21 estados)
n_actions = 3  # 0: bajar volumen, 1: mantener, 2: subir volumen

Q = np.zeros((n_states, n_actions))

alpha = 0.1      # tasa aprendizaje
gamma = 0.9      # factor descuento
epsilon = 0.1    # exploración

def get_reward(spo2):
    if 90 <= spo2 <= 100:
        return 10  # buen estado
    elif spo2 < 90:
        return -10  # peligro
    else:
        return -5  # saturación demasiado alta

def next_spo2(current_spo2, action):
    # Simulación simplificada: acción afecta saturación con algo de ruido
    if action == 0:
        change = -1 + np.random.randn() * 0.5
    elif action == 1:
        change = np.random.randn() * 0.5
    else:
        change = 1 + np.random.randn() * 0.5
    new_spo2 = current_spo2 + change
    return int(np.clip(new_spo2, 80, 100))

episodes = 1000

for ep in range(episodes):
    state = np.random.randint(0, n_states)  # saturación inicial
    done = False
    while not done:
        if np.random.rand() < epsilon:
            action = np.random.choice(n_actions)
        else:
            action = np.argmax(Q[state])

        spo2 = 80 + state  # valor real de saturación
        reward = get_reward(spo2)
        next_state = next_spo2(spo2, action) - 80

        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
        state = next_state

        # Condición de finalización arbitraria
        done = reward == 10

print("Entrenamiento finalizado.")

# Política aprendida (ejemplo para saturación de 85 a 95)
for spo2_level in range(65, 96):
    state = spo2_level - 80
    action = np.argmax(Q[state])
    action_str = {0: "Bajar volumen", 1: "Mantener volumen", 2: "Subir volumen"}[action]
    print(f"Saturación {spo2_level}% -> Acción: {action_str}")


Entrenamiento finalizado.
Saturación 65% -> Acción: Subir volumen
Saturación 66% -> Acción: Subir volumen
Saturación 67% -> Acción: Subir volumen
Saturación 68% -> Acción: Subir volumen
Saturación 69% -> Acción: Subir volumen
Saturación 70% -> Acción: Subir volumen
Saturación 71% -> Acción: Bajar volumen
Saturación 72% -> Acción: Bajar volumen
Saturación 73% -> Acción: Bajar volumen
Saturación 74% -> Acción: Bajar volumen
Saturación 75% -> Acción: Bajar volumen
Saturación 76% -> Acción: Bajar volumen
Saturación 77% -> Acción: Bajar volumen
Saturación 78% -> Acción: Bajar volumen
Saturación 79% -> Acción: Bajar volumen
Saturación 80% -> Acción: Subir volumen
Saturación 81% -> Acción: Subir volumen
Saturación 82% -> Acción: Subir volumen
Saturación 83% -> Acción: Subir volumen
Saturación 84% -> Acción: Subir volumen
Saturación 85% -> Acción: Subir volumen
Saturación 86% -> Acción: Subir volumen
Saturación 87% -> Acción: Subir volumen
Saturación 88% -> Acción: Subir volumen
Saturación 89%

# Optimización de asignación de citas médicas con Q-Learning

In [6]:
import numpy as np

# Estados y acciones
n_states = 11   # pacientes en espera de 0 a 10
n_actions = 4   # agendar de 0 a 3 pacientes

Q = np.zeros((n_states, n_actions))

alpha = 0.1
gamma = 0.95
epsilon = 0.1

def reward(waiting, action):
    # Penalizar si se sobrecarga (más de 10 pacientes en cola)
    next_waiting = waiting + action - 2  # 2 pacientes atendidos por bloque
    if next_waiting < 0:
        next_waiting = 0
    if next_waiting > 10:
        return -10
    # Recompensa mayor si la espera se reduce
    return -next_waiting

def next_state(current, action):
    next_waiting = current + action - 2
    if next_waiting < 0:
        next_waiting = 0
    if next_waiting > 10:
        next_waiting = 10
    return next_waiting

episodes = 500

for ep in range(episodes):
    state = np.random.randint(0, n_states)
    done = False
    steps = 0
    while not done and steps < 20:
        if np.random.rand() < epsilon:
            action = np.random.randint(0, n_actions)
        else:
            action = np.argmax(Q[state])

        r = reward(state, action)
        ns = next_state(state, action)

        Q[state, action] += alpha * (r + gamma * np.max(Q[ns]) - Q[state, action])

        state = ns
        steps += 1

print("Entrenamiento finalizado.")

# Mostrar política aprendida
for waiting in range(n_states):
    best_action = np.argmax(Q[waiting])
    print(f"Pacientes en espera: {waiting} -> Agendar pacientes: {best_action}")


Entrenamiento finalizado.
Pacientes en espera: 0 -> Agendar pacientes: 0
Pacientes en espera: 1 -> Agendar pacientes: 0
Pacientes en espera: 2 -> Agendar pacientes: 0
Pacientes en espera: 3 -> Agendar pacientes: 0
Pacientes en espera: 4 -> Agendar pacientes: 0
Pacientes en espera: 5 -> Agendar pacientes: 0
Pacientes en espera: 6 -> Agendar pacientes: 0
Pacientes en espera: 7 -> Agendar pacientes: 0
Pacientes en espera: 8 -> Agendar pacientes: 0
Pacientes en espera: 9 -> Agendar pacientes: 0
Pacientes en espera: 10 -> Agendar pacientes: 0


# Control de presión arterial con Q-Learning

In [7]:
import numpy as np

# Estados: 0=muy baja, 1=baja, 2=normal, 3=alta, 4=muy alta
n_states = 5
# Acciones: 0=disminuir dosis, 1=mantener dosis, 2=aumentar dosis
n_actions = 3

Q = np.zeros((n_states, n_actions))

alpha = 0.1
gamma = 0.9
epsilon = 0.1

def reward(state):
    # Recompensa alta si está en estado normal (2)
    if state == 2:
        return 10
    else:
        return -abs(2 - state) * 5  # penaliza lo lejos del normal

def next_state(state, action):
    # Simula efecto de la acción
    if action == 0:  # disminuir dosis baja presión
        return max(0, state - 1)
    elif action == 2:  # aumentar dosis sube presión
        return min(n_states - 1, state + 1)
    else:
        # Mantener dosis puede causar pequeña variación aleatoria
        return min(max(0, state + np.random.choice([-1,0,1])), n_states - 1)

episodes = 1000

for ep in range(episodes):
    state = np.random.randint(0, n_states)
    done = False
    steps = 0
    while not done and steps < 20:
        if np.random.rand() < epsilon:
            action = np.random.randint(0, n_actions)
        else:
            action = np.argmax(Q[state])

        r = reward(state)
        ns = next_state(state, action)

        Q[state, action] += alpha * (r + gamma * np.max(Q[ns]) - Q[state, action])

        state = ns
        steps += 1
        if steps >= 20:
            done = True

print("Entrenamiento finalizado.")

# Mostrar política aprendida
for s in range(n_states):
    best_action = np.argmax(Q[s])
    action_name = ["Disminuir dosis", "Mantener dosis", "Aumentar dosis"][best_action]
    print(f"Estado presión arterial: {s} -> Acción recomendada: {action_name}")

Entrenamiento finalizado.
Estado presión arterial: 0 -> Acción recomendada: Aumentar dosis
Estado presión arterial: 1 -> Acción recomendada: Aumentar dosis
Estado presión arterial: 2 -> Acción recomendada: Mantener dosis
Estado presión arterial: 3 -> Acción recomendada: Disminuir dosis
Estado presión arterial: 4 -> Acción recomendada: Disminuir dosis
